In [186]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import math
from Convnet_Functions import *




class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
                nn.Conv3d(1, 32, 5, stride=2),
                nn.ReLU(),
                nn.BatchNorm3d(32),
                nn.LeakyReLU(),
                nn.MaxPool3d(3, stride=2))
        self.conv2 = nn.Sequential(
                nn.Conv3d(32, 64, 5, stride=2),
                nn.ReLU(),
                nn.BatchNorm3d(64),
                nn.LeakyReLU(),
                nn.MaxPool3d(2, stride=2),
                nn.Dropout(0.5))
        self.fc1 = nn.Sequential(
                nn.Linear(64*5*5*6, 64*5*5),
                nn.LeakyReLU(),
                nn.Dropout(0.4))
        self.fc2 = nn.Sequential(
                nn.Linear(64*5*5, 64*5),
                nn.LeakyReLU(),
                nn.Dropout(0.3))
        self.fc3 = nn.Sequential(
                nn.Linear(64*5, 1))

In [2]:
conv1 = nn.Sequential(
    nn.Conv3d(1, 32, [5,5,5], stride=[2,2,2]),
    nn.ReLU(),
    nn.BatchNorm3d(32),
    nn.LeakyReLU(),
    nn.MaxPool3d(3, stride=2))

In [3]:
a =nn.Sequential(
                nn.Conv3d(1, 32, 5, stride=2),
                nn.ReLU(),
                nn.BatchNorm3d(32),
                nn.LeakyReLU(),
                nn.MaxPool3d(3, stride=2))

In [1]:
from network_genetics import *

In [259]:
netparams = gene_aggregator()

In [260]:
netparams.get_conv_shapes()

In [261]:
netparams.get_fc_shapes()

In [262]:
a_gene=conv_layer_params()

In [263]:
a_gene.conv_params

{'kernel': array([3, 5, 4]),
 'stride': array([1, 2, 3]),
 'padding': array([1, 1, 0]),
 'bnorm': False,
 'pool': True,
 'activation1': False,
 'activation2': False,
 'dropout': True,
 'pool_params': {'kernel': array([3, 2, 4]), 'stride': array([3, 3, 1])},
 'dropout_frac': 0.3}

In [5]:
class network_constructor(gene_aggregator):
    
    def __init__(self):
        super().__init__()
        
    
    def build(self):
        
        self.built_convs=[]
        for i in range(self.num_convs):
            self.built_convs.append(self.make_conv_layer(self.convs[i]))
        
        self.built_fc=[]
        for i in range(self.num_fc):
            self.built_fc.append(self.make_fc_layer(self.fcs[i]))
    
    def forward(self,x):
        
        #Run Convs
        for i in range(self.num_convs):
            x = self.built_convs[i](x)
        
        #Flatten and get size
        x = torch.flatten(x, start_dim=1)
        xsh = x.size(-1)
        x = x.view(-1,xsh)
        
        #Run FC
        for i in range(self.num_fc):
            x = self.built_fc[i](x)
        return x
    
    def make_conv_layer(self, params):
        conv = nn.Sequential()

        conv.add_module('conv', nn.Conv3d(params['channels_in'], params['channels_out'], params['kernel'], params['stride'], params['padding']) ) 
        if params['activation1']:
            conv.add_module('activation1', params['activ1_params'])
        if params['bnorm']:
            conv.add_module('BatchNorm', nn.BatchNorm3d(params['channels_out']) )
        if params['activation2']:
            conv.add_module('activation2', params['activ2_params'])
        if params['pool']:
            #I do not know why stride has to be a tuple
            conv.add_module('pool', nn.MaxPool3d(params['pool_params']['kernel'], stride=tuple(params['pool_params']['stride'])) )
        if params['dropout']:
            conv.add_module('dropout', nn.Dropout(params['dropout_frac']) )
        return conv
    
    def make_fc_layer(self, params):
        fc = nn.Sequential()
        
        fc.add_module('linear', nn.Linear(params['features_in'], params['features_out']) )
        if params['activation']:
            fc.add_module('activation', params['activ_params'])
        if params['dropout']:
            fc.add_module('dropout', nn.Dropout(params['dropout_frac']) )
        return fc
    

In [6]:
a = network_constructor()

In [7]:
a.build()

In [8]:
a.forward(

TypeError: forward() missing 1 required positional argument: 'x'

# One class procedurally generated params

#These must be distributable as chromosomes

#The network generator class must be able to assemble any combination of chromosomes into a functioning network

In [17]:
class layer_params(nn.Module):
    
    def __init__(self, shape_in = np.array([106,106,120]) ):
        super().__init__()

        self.shape_in = shape_in
        self.num_conv_layers = random.randint(1,3)
        self.num_fc_layers = random.randint(1,4)
        self.conv_params = self.conv_layer_params()
        self.next_shape = self.get_next_shape()
    
    ##################################################
    ########Functions to generate Parameters##########
    def conv_layer_params(self, channels_in = 1, channels_out = None):

        if not channels_out:
            channels_out = random.randint(20,120)
        kernel =  np.array([ random.randint(2,5), random.randint(2,5), random.randint(2,5) ])
        stride = np.array([ random.randint(1,3), random.randint(1,3), random.randint(1,3) ])
        bnorm = random.choice([True,False])
        pool = random.choice([True,False])
        activation1 = random.choice([True,False])
        activation2 = random.choice([True,False])
        dropout = random.choice([True, False])

        params = { 'channels_in' : channels_in, 'channels_out' : channels_out, 'kernel' : kernel, 'stride' : stride,
                  'bnorm' : bnorm, 'pool' : pool, 'activation1' : activation1, 'activation2' : activation2, 'dropout' : dropout}

        if bnorm:
            params['bnorm_params'] = self.bnorm_layer_params(channels_out)
        if pool:
            params['pool_params'] = self.pool_layer_params()
        if activation1:
            params['activ1_params'] = self.activation_params()
        if activation2:
            params['activ2_params'] = self.activation_params()
        if dropout:
            params['dropout_frac'] = random.randint(2,5)/10

        return params 

    def bnorm_layer_params(self, channels_out = None):
        if not channels_out:
            channels_out = random.randint(20,120)
        params = { 'channels_out' : channels_out }
        return params
    
    def pool_layer_params(self, kernel = None, stride = None):
        if not kernel:
            kernel =  np.array([ random.randint(2,5), random.randint(2,5), random.randint(2,5) ])
        if not stride:
            stride = np.array([ random.randint(1,3), random.randint(1,3), random.randint(1,3) ])

        params = { 'kernel' : kernel, 'stride' : stride }
        return params
    
    def activation_params(self):
        return random.choice([nn.ReLU(), nn.LeakyReLU()])   
    
    def get_next_shape(self, shape_in):
        
        padding=np.array([0])
        pool_kernel = self.conv_params['pool_params']['kernel'] if self.conv_params['pool'] else None
        pool_stride = self.conv_params['pool_params']['stride'] if self.conv_params['pool'] else None

        return Calc_Dims_3d(shape_in, padding, self.conv_params['kernel'], self.conv_params['stride'], self.conv_params['pool'], pool_kernel, pool_stride)

In [ ]:
class network():
    def __init__(self):
        super().__init__()
        
    def __init__(self, shape_in = np.array([106,106,120]) ):
        super().__init__()

        self.shape_in = shape_in
        self.num_conv_layers = random.randint(1,3)
        self.num_fc_layers = random.randint(1,4)
    
    def get_next_shape(self, shape_in):
        
        padding=np.array([0])
        pool_kernel = self.conv_params['pool_params']['kernel'] if self.conv_params['pool'] else None
        pool_stride = self.conv_params['pool_params']['stride'] if self.conv_params['pool'] else None

        return Calc_Dims_3d(shape_in, padding, self.conv_params['kernel'], self.conv_params['stride'], self.conv_params['pool'], pool_kernel, pool_stride)

In [279]:
class procedurally_generated_net(nn.Module, shape_in = np.array([106,106,120])):
    
    def __init__(self):
        super().__init__()
        
    ###############################################
    ########Functions to get output shape##########
    def get_conv_shape_out(self, params, shape = np.array([106,106,120])):
        padding = 0
        kernel = params['kernel']
        stride = params['stride']
        kernel = params['kernel']
        if params['pool']:
            pool_kernel = params['pool_params']['kernel']
            pool_stride = params['pool_params']['stride']
        return Calc_Dims_3d(shape, padding, kernel, stride, pool_kernel, pool_stride)
    
    def get_fc_shape_out(self):
        #placement
        pass

    ##########################################
    ########Functions to make layers##########
    
    def make_conv_layer(params):
        conv = nn.Sequential()

        conv.add_module('conv', nn.Conv3d(params['channels_in'], params['channels_out'], params['kernel'], params['stride']))
        if params['activation1']:
            conv.add_module('activation1', params['activ1_params'])
        if params['bnorm']:
            conv.add_module('BatchNorm', nn.BatchNorm3d(params['bnorm_params']['channels_out']))
        if params['activation2']:
            conv.add_module('activation2', params['activ2_params'])
        if params['pool']:
            conv.add_module('pool', params['pool_params']['kernel'], params['pool_params']['stride'])
        if params['dropout']:
            conv.add_module('dropout', nn.Dropout(params['dropout_frac']))
        return conv

    

TypeError: __init_subclass__() takes no keyword arguments

In [19]:
c1 = layer_params()

In [23]:
c1.conv_params

{'channels_in': 1,
 'channels_out': 64,
 'kernel': array([4, 5, 4]),
 'stride': array([1, 2, 1]),
 'bnorm': True,
 'pool': False,
 'activation1': True,
 'activation2': False,
 'dropout': True,
 'bnorm_params': {'channels_out': 64},
 'activ1_params': ReLU(),
 'dropout_frac': 0.5}

In [14]:
padding=np.array([0])

pool_kernel = c1.conv_params['pool_params']['kernel'] if c1.conv_params['pool'] else None
pool_stride = c1.conv_params['pool_params']['stride'] if c1.conv_params['pool'] else None

Calc_Dims_3d(c1.shape_in, padding, c1.conv_params['kernel'], c1.conv_params['stride'], c1.conv_params['pool'], pool_kernel, pool_stride)

array([53., 35., 41.])

In [63]:
pool_kernel = c1.conv_params['pool_params']['pool_kernel'] if c1.conv_params['pool'] else None

In [13]:
c1.conv_params['pool_params']

{'kernel': array([3, 5, 5]), 'stride': array([2, 1, 2])}

In [18]:
cl1 = make_conv_layer(c1.conv_params)

NameError: name 'make_conv_layer' is not defined

In [171]:
cl1

Sequential(
  (conv): Conv3d(1, 37, kernel_size=[4 3 3], stride=[2 2 3])
  (activation1): LeakyReLU(negative_slope=0.01)
  (activation2): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

In [148]:
c1 = conv_layer_params()

In [149]:
c1

{'channels_in': 1,
 'channels_out': 32,
 'kernel': array([5, 4, 2]),
 'stride': array([1, 2, 3]),
 'bnorm': False,
 'pool': True,
 'activation1': False,
 'activation2': True,
 'pool_params': {'kernel': array([2, 5, 3]), 'stride': array([3, 3, 3])},
 'activ2_params': ReLU(),
 'dropout_frac': 0.5}

In [ ]:
shape_in = np.array([106,106,120])

In [53]:
    a = nn.Sequential()
    
    a.add_module( 'conv', nn.Conv3d(1,32,5,stride=2))
    
    if activation:
        a.add_module( 'activation', nn.ReLU())

In [54]:
a

Sequential(
  (conv): Conv3d(1, 32, kernel_size=(5, 5, 5), stride=(2, 2, 2))
)

In [47]:
        b = nn.Sequential(
                nn.Conv3d(1, 32, 5, stride=2),
                nn.ReLU(),
                nn.BatchNorm3d(32),
                nn.LeakyReLU(),
                nn.MaxPool3d(3, stride=2))

In [49]:
a = nn.Sequential()
a.add_module( 'conv' , nn.Conv3d(1,32,5,stride=2))
a.add_module( 'relu1', nn.ReLU())


TypeError: module name should be a string. Got int

In [25]:
b = make_conv_layer()

TypeError: unsupported operand type(s) for *: 'ReLU' and 'bool'

In [3]:
layer_params = { nn.Conv3d : 1, nn.ReLU() : 1, nn.BatchNorm3d : 1, nn.LeakyReLU() : 1, nn.MaxPool3d : 1}

In [5]:

layer = nn.Sequential(
    layer_params[nn.Conv3d](1,32,5,stride=2)
    )

TypeError: 'int' object is not callable

In [6]:
layer_params[nn.Conv3d]

1